In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [9]:
import pandas as pd

df = pd.read_csv(
    "../data/raw/list_11-09-2025.csv",
    sep=";",
    encoding="utf-8-sig",
    engine="python"
)

df.head()



,identifier,sportName,type,publisherName,publisherIdentifier,title,dateAccepted,number,status,url
0,2025-33/4.1/25,Бочча,"З грифом ""Затверджено""",Міністерство молоді та спорту України,38649881,Правила спортивних змагань з бочча серед осіб ...,11.09.2025,33/4.1/25,Чинний,https://mms.gov.ua/storage/app/sites/16/Sport/...
1,2025-32/3.5/25,Регбі,"З грифом ""Затверджено""",Міністерство молоді та спорту України,38649881,Правила спортивних змагань з регбі,21.08.2025,32/3.5/25,Чинний,https://mms.gov.ua/storage/app/sites/16/Sport/...
2,2025-31/4.1/25,Легка атлетика,"З грифом ""Затверджено""",Міністерство молоді та спорту України,38649881,Правила спортивних змагань з пара легкої атлет...,13.08.2025,31/4.1/25,Чинний,https://mms.gov.ua/storage/app/sites/16/Sport/...
3,2025-30/3.2/25,Кікбоксинг WАКО,"З грифом ""Затверджено""",Міністерство молоді та спорту України,38649881,Правила спортивних змагань з кікбоксингу WAKO,08.08.2025,30/3.2/25,Чинний,https://mms.gov.ua/storage/app/sites/16/Sport/...
4,2025-29/3.2/25,Комбат самозахист ІСО,"З грифом ""Затверджено""",Міністерство молоді та спорту України,38649881,Правила спортивних змагань з комбат самозахист...,08.08.2025,29/3.2/25,Чинний,https://mms.gov.ua/storage/app/sites/16/Sport/...


In [10]:
df.shape

(213, 10)

In [11]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   identifier           213 non-null    str  
 1   sportName            213 non-null    str  
 2   type                 213 non-null    str  
 3   publisherName        213 non-null    str  
 4   publisherIdentifier  213 non-null    int64
 5   title                213 non-null    str  
 6   dateAccepted         212 non-null    str  
 7   number               213 non-null    str  
 8   status               213 non-null    str  
 9   url                  173 non-null    str  
dtypes: int64(1), str(9)
memory usage: 16.8 KB


In [12]:
missing_count = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)

quality_missing = pd.DataFrame({
    "missing_count": missing_count,
    "missing_pct": missing_pct.round(2)
})

quality_missing[quality_missing["missing_count"] > 0]

,missing_count,missing_pct
url,40,18.78
dateAccepted,1,0.47


In [13]:
dup_rows = df.duplicated().sum()
dup_rows
if "identifier" in df.columns:
    df["identifier"].duplicated().sum()
else:
    "Column identifier not found"

In [14]:
required_cols = ["identifier", "sportName", "type", "publisherName", "title", "dateAccepted", "number", "status", "url"]
present = [c for c in required_cols if c in df.columns]
missing_required = [c for c in required_cols if c not in df.columns]

present, missing_required

(['identifier',
  'sportName',
  'type',
  'publisherName',
  'title',
  'dateAccepted',
  'number',
  'status',
  'url'],
 [])

In [15]:
if "dateAccepted" in df.columns:
    dates = pd.to_datetime(df["dateAccepted"], errors="coerce")
    invalid_dates = dates.isna().sum()
    min_date = dates.dropna().min()
    max_date = dates.dropna().max()
    invalid_dates, min_date, max_date
else:
    "Column dateAccepted not found"

In [16]:
if "status" in df.columns:
    df["status"].value_counts(dropna=False)
else:
    "Column status not found"

In [17]:
if "sportName" in df.columns:
    df["sportName"].fillna("NULL").value_counts().head(20)
else:
    "Column sportName not found"

In [18]:
summary = {
    "rows": df.shape[0],
    "cols": df.shape[1],
    "duplicate_rows": int(df.duplicated().sum()),
    "total_missing": int(df.isna().sum().sum()),
}

if "identifier" in df.columns:
    summary["identifier_missing"] = int(df["identifier"].isna().sum())
    summary["identifier_duplicates"] = int(df["identifier"].duplicated().sum())

if "dateAccepted" in df.columns:
    dates = pd.to_datetime(df["dateAccepted"], errors="coerce")
    summary["dateAccepted_invalid"] = int(dates.isna().sum())
    summary["dateAccepted_min"] = None if dates.dropna().empty else str(dates.dropna().min().date())
    summary["dateAccepted_max"] = None if dates.dropna().empty else str(dates.dropna().max().date())

pd.DataFrame([summary])

,rows,cols,duplicate_rows,total_missing,identifier_missing,identifier_duplicates,dateAccepted_invalid,dateAccepted_min,dateAccepted_max
0,213,10,0,41,0,0,120,2013-01-04,2025-11-09


Набір даних містить 213 записів і 10 атрибутів, що описують затверджені правила спортивних змагань, опубліковані Міністерством молоді та спорту України. Структура даних коректна: всі обов’язкові поля присутні, дублікати записів відсутні, ідентифікатори унікальні.

Виявлено незначні проблеми якості даних. Найбільше пропусків у полі url (≈18.8%), що означає відсутність посилання на документ для частини записів. У полі dateAccepted є 1 пропуск і значна кількість некоректно розпізнаних дат через різні формати запису, що потребує нормалізації перед часовим аналізом.

Діапазон дат охоплює період з 2013-01-04 до 2025-11-09. Поле status однорідне (усі записи мають значення «Чинний»), що свідчить про відсутність історичних або архівних документів у вибірці.

Загалом датасет придатний для подальшого аналізу після попереднього очищення: нормалізації формату дат і, за потреби, обробки відсутніх URL.